In [4]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from openai import OpenAI
from langchain.prompts import ChatPromptTemplate

from llama_parse import LlamaParse
import nest_asyncio

import pandas as pd
import os
import csv
import time

In [ ]:
# Secret:

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = 
os.environ['OPENAI_API_KEY'] = 
os.environ['LLAMA_CLOUD_API_KEY'] = 

In [6]:
# parse the document, save as a md file.
nest_asyncio.apply()


parser = LlamaParse(
    #api_key=,  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    num_workers=4,  # if multiple files passed, split in `num_workers` API calls
    verbose=True,
    language="en",  # Optionally you can define a language, default=en
)

# sync
documents = parser.load_data("Orientation Documents-Spring 2024 Orientation Document.pdf")
documents

# save the parsed data
parse_path = "parsed_data.md"
with open(parse_path, "w") as f:
    f.write(documents[0].text)

Started parsing the file under job_id cac11eca-fa9d-40ca-8237-e768bc5a9ea7


In [9]:
# load the parsed data
document_path = "parsed_data.md"
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

In [10]:
# Split the parse data into chunks, the chunk size is 512 characters
# This parameter is very important, it will affect the performance of the model
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=20)
splits = text_splitter.split_documents(loaded_documents)

# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [12]:
#### RETRIEVAL and GENERATION using ChatGPT 3.5####

# pull a generic prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain_ChatGPT = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain_ChatGPT.invoke("What is this program?")

"The program mentioned involves selecting a concentration or specialization for your current program of study. It is expected that students in this program are proficient in multiple programming languages and have taken advanced topics like Advanced OS, Networking, Theory, and/or Algorithms. More information can be found on the program's website."

In [13]:
#### RETRIEVAL and GENERATION using Llama###

# The Llama model is hosted locally and can be accessed via the OpenAI API

template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Don't state your role and task. Avoid using preface like "According to the provided context".

Question: {question}

Context: {context}
"""

local_prompt = ChatPromptTemplate.from_template(template)
local_prompt


#local_prompt = prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the function to interact with the local LLM server
def local_llm(prompt_text):
    # Point to the local server
    client = OpenAI(base_url="http://192.168.68.78:1234/v1", api_key="lm-studio")

    completion = client.chat.completions.create(
    model="lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF",
    messages=[
        {"role": "user", "content": str(prompt_text)}
    ],
    temperature=0.8,
    )
    return str(completion.choices[0].message.content)

# Update the chain to use the local LLM server
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | local_prompt
    | local_llm
    | StrOutputParser()
)

In [15]:
# load evaluation questions
questions = pd.read_csv("faq_output.csv")
questions

,category,qns_no,qns,ans
0,New Student Onboarding/GT Email Account,1,Where do I send my official transcript(s) and ...,Send your final official transcript(s) and oth...
1,New Student Onboarding/GT Email Account,2,What happens if I do not submit my official tr...,"If you are a new student starting Spring 2024,..."
2,New Student Onboarding/GT Email Account,3,Is there an orientation for the OMSCS program?,"Since this is an online program, we do not hav..."
3,New Student Onboarding/GT Email Account,4,What are the expectations regarding my GT emai...,"The Institute, as well as the department, will..."
4,New Student Onboarding/GT Email Account,5,How can I access my GT email account?,Please visit this website for information on h...
...,...,...,...,...
153,Graduation,13,Is it possible to graduate with honors/distinc...,Graduating with distinction/honors is not avai...
154,Graduation,14,Can I continue taking OMSCS courses after I gr...,It is possible for students to continue taking...
155,Graduation,15,Can I continue being a TA after I graduate fro...,"This is possible, but it is more complicated t..."
156,Graduation,16,What technology-related services will I retain...,Please visit https://gatech.service-now.com/ho...


In [16]:
# This is a random question generator
def get_sample_questions():
    random_question = questions.sample().iloc[0]
    return {'question':random_question.qns, 'answer':random_question.ans}

print(get_sample_questions())

{'question': 'How long should I expect to wait before I receive a wait list notification?', 'answer': 'There is no specific amount of time as to when students will receive a wait list notification, as we unfortunately\ncannot guarantee that everyone on every wait list will get into the course. As stated previously, some courses are\nunable to scale above a certain maximum while still providing an effective learning experience for students.\nPlease note that the advisors are unable to determine/predict which courses will have more seats added.\nTherefore, please be sure to monitor your email account carefully and frequently, including your spam folder, in\ncase you receive a wait list notification.'}


In [17]:
# Get sample questions and invoke the chains once

start_time = time.time()
q_and_a = get_sample_questions()
print('The question is:', q_and_a.get('question'))
print()

# Invoke chatGPT
start_time = time.time()
print('chatGPT answer:', rag_chain_ChatGPT.invoke(q_and_a.get('question')))
print('Time to invoke chatGPT:', round(time.time() - start_time,2),'seconds')
print()

# Invoke local LLAMA
start_time = time.time()
print('Local LLAMA answer:', rag_chain.invoke(q_and_a.get('question')))
print('Time to invoke local LLAMA:', round(time.time() - start_time,2),'seconds')
print()

print('The official answer is:', q_and_a.get('answer'))

The question is: If I am not sure that I will meet the degree requirements by the end of the term, am I still permitted to

chatGPT answer: If you do not meet your foundational requirement by the designated deadline, your graduation application may be inactivated. In this case, you would need to enroll in a future term to fulfill the degree requirements.
Time to invoke chatGPT: 1.17 seconds

Local LLAMA answer: No, you are not permitted to attend the commencement ceremony if you're unsure about meeting degree requirements. You can still attend, but you'll need to enroll in a future term to fulfill the degree requirements.
Time to invoke local LLAMA: 1.42 seconds

The official answer is: attend the commencement ceremony?
Since grades are not finalized until after the commencement ceremony, there have been times when students
Click here to return to Table of Contents
34


have attended the commencement ceremony but unfortunately did not earn the grade(s) they needed. In this
case, the st

In [19]:
count = 0

with open('faq_output_with_answers_compare_new.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Question', 'Answer', 'chatGPT_answer', 'Local_LLAMA_answer','chat_gpt_time','local_llama_time'])

    for row in questions.iterrows():
        question = row[1].qns
        answer = row[1].ans
        start_time = time.time()
        #chatGPT_answer = rag_chain_ChatGPT.invoke(question)
        chatGPT_answer = ''
        chat_gpt_time = round(time.time() - start_time,2)

        start_time = time.time()
        Local_LLAMA_answer = rag_chain.invoke(question)
        local_llama_time = round(time.time() - start_time,2)

        print(f'Finished generating answer for question {count}.')
        count+=1

        # write the row to the CSV file
        writer.writerow([question, answer, chatGPT_answer, Local_LLAMA_answer, chat_gpt_time, local_llama_time])

Finished generating answer for question 0.
Finished generating answer for question 1.
Finished generating answer for question 2.
Finished generating answer for question 3.
Finished generating answer for question 4.
Finished generating answer for question 5.
Finished generating answer for question 6.
Finished generating answer for question 7.
Finished generating answer for question 8.
Finished generating answer for question 9.
Finished generating answer for question 10.
Finished generating answer for question 11.
Finished generating answer for question 12.
Finished generating answer for question 13.
Finished generating answer for question 14.
Finished generating answer for question 15.
Finished generating answer for question 16.
Finished generating answer for question 17.
Finished generating answer for question 18.
Finished generating answer for question 19.
Finished generating answer for question 20.
Finished generating answer for question 21.
Finished generating answer for question 22